<a href="https://colab.research.google.com/github/jaisal1024/advanced-investments-final/blob/master/AI_final_data_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Investments Finals
Data is collected from CRSP and OptionMetric via WRDS

In [0]:
import sys
import os
import pandas as pd
import datetime
from datetime import datetime
from google.colab import files

In [0]:
# import raw data from github
url_crsp = 'https://raw.githubusercontent.com/jaisal1024/advanced-investments-final/master/crsp-monthly.csv'
url_fama = 'https://raw.githubusercontent.com/jaisal1024/advanced-investments-final/master/fama-french-indices.csv'
url_opt = 'https://raw.githubusercontent.com/jaisal1024/advanced-investments-final/master/daily-options-vol.csv'
df_crsp = pd.read_csv(url_crsp)
df_fama = pd.read_csv(url_fama)
df_opt = pd.read_csv(url_opt)

## Add headers to options data 
Datetime and datemonth fields

In [0]:
df_opt['datetime'] = df_opt['date'].apply(lambda x: datetime.strptime(str(x), '%Y%m%d'))

In [0]:
df_opt['datemonth'] = df_opt['date'].apply(lambda x: str(x)[:-2])

## Create Call and Put dataframes
Then sort them and reset the indices

In [0]:
df_opt_c = df_opt.loc[df_opt['cp_flag'] == 'C']
df_opt_p = df_opt.loc[df_opt['cp_flag'] == 'P']
df_opt_c = df_opt_c.sort_values(['ticker', 'datemonth']).reset_index(drop='True')
df_opt_p = df_opt_p.sort_values(['ticker', 'datemonth']).reset_index(drop='True')

## Group by ticket and datemonth and then calculate the sums
Then rename the volume and open interest columns to prep for merge


In [0]:
df_opt_p_month = df_opt_p.groupby(['ticker', 'datemonth']).sum()

In [0]:
df_opt_c_month = df_opt_c.groupby(['ticker', 'datemonth']).sum()

In [0]:
df_opt_c_month.rename(index = str, columns = {'volume': 'call_vol', 'open_interest':'call_open_interest'}, inplace=True)
df_opt_p_month.rename(index = str, columns = {'volume': 'put_vol', 'open_interest':'put_open_interest'}, inplace=True)

## Concatenate them into the final options dataframe
drop useless columns

In [0]:
df_opt_final = pd.concat([df_opt_c_month, df_opt_p_month], axis=1)

In [0]:
df_opt_final = df_opt_final.drop(columns = ['index_flag', 'date', 'secid'])

## Create ratio for call/put vol and call/put open interest

In [0]:
df_opt_final['ratio_vol'] = df_opt_final['call_vol']/df_opt_final['put_vol']

In [0]:
df_opt_final['ratio_open_interest'] = df_opt_final['call_open_interest']/df_opt_final['put_open_interest']

## Rename ARNP stock to AA in dataset
sort then reindex

In [0]:
#rename ARNP to AA in dataset
df_opt_final.loc[df_opt_final['ticker'] == 'ARNC', 'ticker'] = 'AA'

In [0]:
df_opt_final = df_opt_final = df_opt_final.sort_values(['ticker', 'datemonth']).reset_index()
df_opt_final.head()

,level_0,index,ticker,datemonth,call_vol,call_open_interest,put_vol,put_open_interest,ratio_vol,ratio_open_interest
0,0,720,AA,200607,303888,6556565,167897,4251089,1.809967,1.542326
1,1,721,AA,200608,146724,6937698,135537,4686256,1.082538,1.480435
2,2,722,AA,200609,184089,6551174,242312,5005516,0.759719,1.308791
3,3,723,AA,200610,352148,7639471,193484,5178574,1.820037,1.475207
4,4,724,AA,200611,174649,5975063,203050,4179924,0.860128,1.429467


## Export to CSV and download 

In [0]:
df_opt_final.to_csv('monthly_options_final.csv')

In [0]:
files.download("monthly_options_final.csv")

## Merge CRSP dataset and Options dataset
create datemonth column in crsp data
rename TICKER to ticker
make sure type of ticker and datemonth are string
merge on inner join

In [0]:
# merge options data and crsp data
df_crsp['datemonth'] = df_crsp['date'].apply(lambda x: str(x)[:-2])

In [0]:
df_crsp = df_crsp.sort_values(['TICKER', 'datemonth']).reset_index(drop='True')
df_crsp.rename(index = str, columns = {'TICKER': 'ticker'}, inplace=True)
df_crsp.head()

In [0]:
df_opt_final.ticker = df_opt_final.ticker.astype(str)
df_opt_final.datemonth = df_opt_final.datemonth.astype(str)

In [0]:
df_crsp_final.ticker = df_crsp_final.ticker.astype(str)
df_crsp_final.datemonth = df_crsp_final.datemonth.astype(str)

In [0]:
df_crsp_final = df_crsp
df_crsp_final = df_crsp_final.merge(df_opt_final, on = ['ticker', 'datemonth'], how = 'inner')

In [0]:
finals_tick = list(df_crsp_final.ticker.unique())

In [0]:
# df_crsp_final.to_csv('crsp_merged.csv')
# files.download("crsp_merged.csv")

## Find Missing Data

In [0]:
opt_tick = list(df_opt_final.ticker.unique())
crsp_tick = list(df_crsp.ticker.unique())

In [0]:
print(len(crsp_tick), len(opt_tick), len(finals_tick))

98 83 81


In [0]:
list(set(opt_tick).difference(finals_tick))

['NSM', 'BUD']

In [0]:
list(set(crsp_tick).difference(finals_tick))

['BHI',
 'MLSS',
 'MEDI',
 'CVX',
 'CSC',
 'TYC',
 'DOW',
 'UPS',
 'nan',
 'OMX',
 'SNSTA',
 'MSS',
 'HSH',
 'SLE',
 'EMC',
 'DD',
 'LTD']

# Merge fama french risk free rate

In [0]:
# merge risk free rate
df_fama_temp = df_fama.filter(['dateff','rf'], axis=1)
df_fama_temp = df_fama_temp.rename(index = str, columns = {'dateff' : 'date'})
df = pd.merge(df_crsp_final, df_fama_temp, on = 'date', how = 'left')

In [0]:
df.loc[df['RET']== 'C', 'RET'] = '0'
df['RET'] = df['RET'].astype(float)
df.dtypes
df['RET-RF'] = df['RET'] - df['rf']

In [0]:
df.to_csv('crsp_merged-81.csv')
files.download("crsp_merged-81.csv")